### Ramsey multiplicity minimisation

This note computes 
$$
  \min_G t(H, G) + t(H, 1 - G)
$$
where 1-G is the complement of graph G.

In [ ]:
import flag_algebra.flag_algebra as fa
import flag_algebra.build_problem as bp
import networkx as nx
import cvxpy as cp
import math
import numpy as np

In [ ]:
# Current implementation does not support t(H, 1-G).
# So we need to manually compute this. 

def compute_hom_complement_coefficient(H, F):
  matcher = nx.isomorphism.GraphMatcher(F, nx.complement(H))
  num_monomorphisms = sum(1 for _ in matcher.subgraph_monomorphisms_iter())

  return num_monomorphisms / math.perm(len(F.nodes), len(H.nodes))

def compute_hom_complement_coefficients(H, atlas=None):
  if atlas is None:
    atlas = fa.get_graph_atlas(len(H.nodes))
  
  res = []
  for g in atlas:
    coeff = compute_hom_complement_coefficient(H, g)
    res.append(coeff)
  return np.array(res)

In [ ]:
H = nx.wheel_graph(5)

comp_coeff = compute_hom_complement_coefficients(H, fa.get_graph_atlas(7))

problem, variable_dict = bp.build_problem(
  objectives = [('hom', H, 1), ('hom', H, 1, comp_coeff)],
  constraints = [],
  sdp_configs = [(4, 1), (5, 3), (6, 5)],
  lowerbound=True,
)

problem.solve(solver=cp.SCS)

print("Optimal value:", problem.value)
print("Naive upper bound", 2 * (0.5) ** len(H.edges))